### Model Training

In [35]:
import pandas as pd

In [56]:
df =  pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [57]:
df = df.drop(labels=['id'],axis=1)

In [58]:
# Independent and Dependent Features
X  = df.drop(labels=['price'],axis=1)
Y = df['price']

In [59]:
Y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [60]:
categorical_cols = X.columns[X.dtypes==object]
numerical_cols = X.columns[X.dtypes!=object]

In [61]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [62]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [63]:
# WORST TO BEST
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['J','I','H','G','F','E','D']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [64]:
from sklearn.impute import SimpleImputer # Handle missing values
from sklearn.preprocessing import StandardScaler # Handle Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Handle Ordinal Encoding
from sklearn.pipeline import Pipeline # Pipeline
from sklearn.compose import ColumnTransformer # Combine different technique

In [65]:
num_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [66]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [67]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [68]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,-0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,-0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,-1.526720,-0.648127


In [69]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [70]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [71]:
lr_preds = lr.predict(X_test)
lr_report = {'R2 Score':r2_score(y_test,lr_preds),'Mean Absolute Error':mean_absolute_error(y_test,lr_preds),'Mean Squared Error':mean_squared_error(y_test,lr_preds)}
print(lr_report)

{'R2 Score': 0.9362906819996049, 'Mean Absolute Error': 675.0758270067483, 'Mean Squared Error': 1029473.3531156846}


In [72]:
lr.coef_

array([ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
        -494.17005097,   -76.32351645,    68.80035873,   464.67990411,
         652.10059539])

In [73]:
lr.intercept_

3976.878738902296

In [74]:
def evaluate_model(preds,truth):
    mae = mean_absolute_error(preds,truth)
    mse = mean_squared_error(preds,truth)
    r2 = r2_score(preds,truth)
    return mae,mse,r2

In [77]:
## Training Multiple Models
models = {
    'Linear': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet() 
}
for i in models:
    model = models[i]
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    mae,mse,r2 = evaluate_model(preds,y_test)
    print(i)
    print('MAE',mae)
    print('MSE',mse)
    print('R2',r2*100)
    print('='*30)
    print('\n')

Linear
MAE 675.0758270067483
MSE 1029473.3531156846
R2 93.14497641619309


Lasso
MAE 676.2421173665508
MSE 1029533.1506505491
R2 93.1370638880677


Ridge
MAE 675.1077629781329
MSE 1029482.8101268952
R2 93.14462925139522


ElasticNet
MAE 1060.9432977143008
MSE 2351174.8713978743
R2 77.8554360252738


